# Model 5 - CNN with trainable embeddings: This models consists of a Convolutional Neural Network (CNN) with window size 4, updating the embeddings during training.

### The graph is shown below 
![graph can't be found](../data/images/graph_cnn.png)

In [20]:

from keras.layers import Embedding
import pickle
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv1D, MaxPooling1D
from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional
from keras import layers
from keras import backend as K
import keras
import tensorflow as tf
from tensorflow.contrib.tensorboard.plugins import projector
import logger_single
from logger_single import NBatchLogger

In [4]:
trainX,trainY,testX,testY,X_test,num_vocab, features=pickle.load(open('./twitter-datasets/train_test_prep_expand_full','rb'))


In [5]:
trainX=np.concatenate([trainX,testX[:485000]])
trainY=np.concatenate([trainY,testY[:485000]])
testX=testX[485000:]
testY=testY[485000:]

In [6]:
embeddings_index = {}
f = open( './twitter-datasets/glove.840B.300d.txt')
for line in f:
    values = line.split()
    word = values[0]
    try:
        coefs = np.asarray(values[1:], dtype='float64')
        embeddings_index[word] = coefs
    except:
        pass
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 2195885 word vectors.


In [7]:
mult_char=dict()

for char_ in 'abcdefghijklmnopqrstuvwxyz':
    mult_char[char_]=char_*3
embedding_matrix = np.random.random((num_vocab,300))
feature_not_present=[]
for i,feature in enumerate(features):
    feature=feature.replace('#','') 
    feature=feature.replace('"',"")
    feature=feature.replace("'","")
    feature=feature.replace("-","")
    feature=feature.replace("_","")
    for key,value in mult_char.items():
        feature = feature.replace(value,key)
    feature=feature.replace('#','') 
    embedding_vector=embeddings_index.get(feature)
    if(embedding_vector is not None):
        embedding_matrix[i] = embedding_vector
    else:
        feature_not_present.append(feature)
del(embeddings_index) # free up memory 

In [ ]:
def save_metadata():
    # Use the same LOG_DIR where you stored your checkpoint.
    summary_writer = tf.summary.FileWriter('./Graph/')
    # Format: tensorflow/contrib/tensorboard/plugins/projector/projector_config.proto
    config = projector.ProjectorConfig()
    # You can add multiple embeddings. Here we add only one.
    embedding = config.embeddings.add()
    embedding.tensor_name = 'embedded_layer/W'
    # Link this tensor to its metadata file (e.g. labels).
    pd.Series(features).to_csv('./Graph/metadata.tsv',sep='\n',index=False,header=False)
    embedding.metadata_path = './Graph/metadata.tsv'
    # Saves a configuration file that TensorBoard will read during startup.
    projector.visualize_embeddings(summary_writer, config)
    

save_metadata()

In [31]:
keras.backend.clear_session()
embedding_layer = Embedding(num_vocab,
                            300,
                            weights=[embedding_matrix],
                            input_length=100,
                            trainable=True,name='embedding1')

tbCallBack = keras.callbacks.TensorBoard(log_dir='./Graph', histogram_freq=0, write_graph=True, write_images=True,\
                                          embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=features)

In [ ]:

checkpointer = keras.callbacks.ModelCheckpoint(filepath='./Graph/weightscnn_2.hdf5', verbose=1, save_best_only=True)

In [43]:

callback_save=NBatchLogger('cnn_vary_embed_2',testX,testY,display=1000,if_exist='replace')


#callback_save=NBatchLogger('cnn_lstm_1',testX,testY,display=1000,if_exist='append')
reg=keras.regularizers.l2(0.01)
sequence_input = layers.Input(shape=(100,), dtype='int32',name='main_input')
embedded_sequences = embedding_layer(sequence_input)
net1 = Conv1D(2, 3, activation='relu',name = 'conv1_1')(embedded_sequences)
#net1 = layers.Dropout(0.2,name = 'dropout1_1')(net1)
net1 = MaxPooling1D(3,strides=1,name = 'maxpool1_1')(net1)
net1 = layers.Dropout(0.2,name = 'dropout1_1')(net1)
net1 = Flatten(name = 'flatten1')(net1)
net1 = Dense(128, activation='relu',name = 'dense1_1',activity_regularizer=reg)(net1)
net1 = layers.Dropout(0.2,name = 'dropout1_3')(net1)


In [37]:
'''
sequence_input_2 = layers.Input(shape=(100,), dtype='int32',name='main_input2')
net2 = embedding_layer(sequence_input_2)
net2 = Bidirectional(LSTM(256,return_sequences=True),name= 'bidir_LSTM')(net2)
net2 = layers.LSTM(64,name='LSTM_2')(net2)
net2 = layers.Dropout(0.3,name='dropout2_1')(net2)
net2 = Dense(128, activation='relu',name='dense2_1')(net2)
'''

"\nsequence_input_2 = layers.Input(shape=(100,), dtype='int32',name='main_input2')\nnet2 = embedding_layer(sequence_input_2)\nnet2 = Bidirectional(LSTM(256,return_sequences=True),name= 'bidir_LSTM')(net2)\nnet2 = layers.LSTM(64,name='LSTM_2')(net2)\nnet2 = layers.Dropout(0.3,name='dropout2_1')(net2)\nnet2 = Dense(128, activation='relu',name='dense2_1')(net2)\n"

In [38]:
#merged = layers.add(net1,name='merged')

In [39]:
merged = layers.Dropout(0.3,name='dropout2_1')(net1)
merged = Dense(128, activation='relu',reg,name='dense_final')(merged)
preds = layers.Dense(2, activation='softmax',name='output')(merged)

model = keras.Model(sequence_input, preds)
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])




In [40]:
print('Training')
model.fit(trainX, trainY,
          batch_size=64,
          epochs=10,
          validation_data=(testX,testY),callbacks=[checkpointer,tbCallBack,callback_save],verbose=1)

Training
Train on 2485000 samples, validate on 15000 samples
Epoch 1/10
  36160/2485000 [..............................] - ETA: 28:47 - loss: 0.5196 - acc: 0.7177

KeyboardInterrupt: 